In [1]:
import pickle
from neuron_visualization.NeuronVisualizer import NeuronVisualizer
import pandas as pd

with open("models/triangle_rnn_model.pkl", "rb") as f:
    model = pickle.load(f)
model.set_device("cpu")
visualizer = NeuronVisualizer(model)
data = pd.read_csv("data/triangle_env_data.csv")
data = data[data['run_id'].isin([0, 1])]
print(data.shape)
visualizer.retrieve_activations(data, use_predicted=True)


(71, 11)


100%|██████████| 2/2 [00:00<00:00, 72.69it/s]


In [ ]:
maps = visualizer.get_spatial_maps()
print(visualizer.compute_cos_sim_on_maps(maps))

AttributeError: 'str' object has no attribute 'write'